In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from torch_geometric.data import Data,Dataset

total_parms_new = pd.read_csv('/mnt/ssd1/sunyifan/WorkStation/dpuf/my_DPGCN/total_params_new.csv',index_col=0)
total_parms = pd.read_csv('/mnt/ssd1/sunyifan/WorkStation/dpuf/my_DPGCN/total_params.csv',index_col=0)

from typing import List, Literal, Sequence, Union


dataset_list = ['cora','citeseer','pubmed','computers','photo','cs','physics','reddit','github','flickr','lastfmasia']

def get_datasetwise_df(df:pd.DataFrame,name=None):
    if not name:
        dflist = []
        for i in ['cora','citeseer','pubmed','computers','photo','cs','physics','reddit','github','flickr','lastfmasia']:
            dflist.append(df[df['Dataset']==i])
        return dflist
    else:
        return df[df['Dataset']==name]

def baseline(df:pd.DataFrame):
    clean_adam = df[(df['Dp']==False)&(df['Rdp']==True)&(df['Ldp']==False)&(df['Optim type']=='adam')]
    clean_sgd = df[(df['Dp']==False)&(df['Rdp']==True)&(df['Ldp']==False)&(df['Optim type']=='sgd')]
    return clean_adam,clean_sgd

def rdp_drop(df:pd.DataFrame):
    labels = ['Vanilla train acc','Vanilla train loss','Vanilla train f1',
          'Shadow train acc', 'Shadow train loss',
       'Shadow train f1', 'Shadow test acc', 'Shadow test loss',
       'Shadow test f1', 'MIA subsample rate','MIA seed','Shadow runtime per', 'Num val', 'Num test', 'Layers', 'Hidden dims','Activation','Shadow learning rate','Dropout','Sampler','Dp','Rdp','Ldp']
    return df.drop(columns=labels)

def get_res_rdp(cora:pd.DataFrame):
    maxx = ['Vanilla test acc','Vanilla test f1']
    meann = ['Vanilla runtime per','MIA mlp', 'MIA svm','MIA ranfor', 'MIA logi', 'MIA ada', 'MIA confidence mse','Epsilon','Delta']
    minn = ['Epsilon','Delta']
    res = {}
   
    for k in maxx:
        res[k] = cora.max()[k]
    for k in meann:
        res[k] = cora.mean(numeric_only=True)[k]
    for k in minn:
        res[k] = cora.mean(numeric_only=True)[k]
    return res
    

total_parms_new.columns

/mnt/ssd1/sunyifan/conda/mygcn/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Index(['Dataset', 'Vanilla train acc', 'Vanilla train loss',
       'Vanilla train f1', 'Vanilla test acc', 'Vanilla test loss',
       'Vanilla test f1', 'Shadow train acc', 'Shadow train loss',
       'Shadow train f1', 'Shadow test acc', 'Shadow test loss',
       'Shadow test f1', 'MIA subsample rate', 'MIA mlp', 'MIA svm',
       'MIA ranfor', 'MIA logi', 'MIA ada', 'MIA confidence mse',
       'MIA confidence thr', 'MIA seed', 'Vanilla runtime per',
       'Shadow runtime per', 'Epsilon', 'Delta', 'Dp', 'Rdp', 'Ldp',
       'Norm bound', 'Noise scale', 'Sampler', 'Sampler batchsize',
       'Occurance k', 'Cluster numparts', 'Saint rootnodes',
       'Saint samplecoverage', 'Saint walklenth', 'Epochs', 'Shadow epochs',
       'Num val', 'Num test', 'Layers', 'Hidden dims', 'Learning rate',
       'Shadow learning rate', 'Dropout', 'Activation', 'Early stopping',
       'Patience', 'Optim type'],
      dtype='object')

'cora','citeseer','pubmed','computers','photo','cs','physics','reddit','github','flickr','lastfmasia'

In [30]:
datadfs = get_datasetwise_df(total_parms_new)
cora,_ = baseline(datadfs[6])
cora['Sampler'].drop_duplicates()

640    cluster
Name: Sampler, dtype: object

In [3]:
cora_ad_d = cora[(cora['Sampler']=='occurance')]
cora_ad_d['Sampler batchsize'].drop_duplicates()

1491    0.2
Name: Sampler batchsize, dtype: float64

In [4]:
cora_ad_d['Noise scale'].drop_duplicates()

1491    1.0
Name: Noise scale, dtype: float64

In [5]:
cora_ad_d['Norm bound'].drop_duplicates()

1491    2.0
Name: Norm bound, dtype: float64

In [6]:
cora_ad_d['Occurance k'].drop_duplicates()

1491    10
Name: Occurance k, dtype: int64

In [7]:
get_res_rdp(cora_ad_d)

{'Vanilla test acc': 0.4307,
 'Vanilla test f1': 0.4307,
 'Vanilla runtime per': 2.86062,
 'MIA mlp': 0.51176,
 'MIA svm': 0.51212,
 'MIA ranfor': 0.5119,
 'MIA logi': 0.5009399999999999,
 'MIA ada': 0.51406,
 'MIA confidence mse': 0.5382100000000001,
 'Epsilon': 32.041760000000004,
 'Delta': 0.0}

In [8]:
df = pd.read_csv('/mnt/ssd1/sunyifan/WorkStation/dpuf/my_DPGCN/ress/cs_ns2_nb1_occurance.csv',index_col=0)
df['Sampler batchsize'].drop_duplicates()

0    0.2
Name: Sampler batchsize, dtype: float64

In [9]:
df['Noise scale'].drop_duplicates()

0    1.0
Name: Noise scale, dtype: float64

In [10]:
df['Occurance k'].drop_duplicates()

0    10
1    20
Name: Occurance k, dtype: int64

In [11]:
od1 = df[(df['Occurance k']==20)&(df['Sampler batchsize']==.2)&(df['Noise scale']==1)]
get_res_rdp(od1)

{'Vanilla test acc': 0.918,
 'Vanilla test f1': 0.918,
 'Vanilla runtime per': 1.7381571428571427,
 'MIA mlp': 0.5133142857142857,
 'MIA svm': 0.5102857142857143,
 'MIA ranfor': 0.5093142857142857,
 'MIA logi': 0.5116999999999999,
 'MIA ada': 0.5127571428571428,
 'MIA confidence mse': 0.518957142857143,
 'Epsilon': 26.853457142857145,
 'Delta': 0.0}

In [26]:
df=pd.read_csv('/mnt/ssd1/sunyifan/WorkStation/dpuf/my_DPGCN/ress/cs_ns2_nb1_neighbor.csv',index_col=0)
df['Sampler batchsize'].drop_duplicates()

0    0.2
Name: Sampler batchsize, dtype: float64

In [27]:
df['Noise scale'].drop_duplicates()

0    2.0
Name: Noise scale, dtype: float64

In [28]:
get_res_rdp(df[(df['Sampler batchsize']==.2)&(df['Noise scale']==2)])

{'Vanilla test acc': 0.9134,
 'Vanilla test f1': 0.9134,
 'Vanilla runtime per': 2.0372,
 'MIA mlp': 0.5219,
 'MIA svm': 0.5209,
 'MIA ranfor': 0.5096,
 'MIA logi': 0.5119,
 'MIA ada': 0.5154,
 'MIA confidence mse': 0.5286,
 'Epsilon': 35.8937,
 'Delta': 0.0}